# Perkenalan

Nama/Batch : Sandika/008

# Import Pustaka

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder

# Data Loading

In [ ]:
df_dummy= pd.read_csv('./data/rideshare_kaggle.csv')
df_dummy.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800


# EDA

In [ ]:
df=df_dummy.copy()

In [ ]:
df.isnull().sum() #Mencari jumlah data dengan nilai NaN

id                                 0
timestamp                          0
hour                               0
day                                0
month                              0
datetime                           0
timezone                           0
source                             0
destination                        0
cab_type                           0
product_id                         0
name                               0
price                          55095
distance                           0
surge_multiplier                   0
latitude                           0
longitude                          0
temperature                        0
apparentTemperature                0
short_summary                      0
long_summary                       0
precipIntensity                    0
precipProbability                  0
humidity                           0
windSpeed                          0
windGust                           0
windGustTime                       0
v

saya akan menggunakan linear regression dimana nilai Y= 'Price'

In [ ]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637976 entries, 0 to 693070
Data columns (total 57 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           637976 non-null  object 
 1   timestamp                    637976 non-null  float64
 2   hour                         637976 non-null  int64  
 3   day                          637976 non-null  int64  
 4   month                        637976 non-null  int64  
 5   datetime                     637976 non-null  object 
 6   timezone                     637976 non-null  object 
 7   source                       637976 non-null  object 
 8   destination                  637976 non-null  object 
 9   cab_type                     637976 non-null  object 
 10  product_id                   637976 non-null  object 
 11  name                         637976 non-null  object 
 12  price                        637976 non-null  float64
 13 

In [ ]:
df.groupby('name').mean()['price'].sort_values(ascending=False)

name
Lux Black XL    32.324086
Black SUV       30.286763
Lux Black       23.062468
Black           20.523786
Lux             17.771240
UberXL          15.678144
Lyft XL         15.309363
UberX            9.765074
WAV              9.765019
Lyft             9.610885
UberPool         8.752500
Shared           6.029893
Name: price, dtype: float64

In [ ]:
df.groupby('cab_type').mean()['price'].sort_values(ascending=False)

cab_type
Lyft    17.351396
Uber    15.795343
Name: price, dtype: float64

# Data Preprocessing

In [ ]:
# Proses mencari korelasi
df.corr()

,timestamp,hour,day,month,price,distance,surge_multiplier,latitude,longitude,temperature,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
timestamp,1.000000,-0.028501,-0.341420,0.771245,0.000808,0.003290,0.000611,0.168295,-0.136716,-0.249027,...,-0.211887,0.999364,-0.307194,0.998195,-0.186495,0.999290,-0.378538,0.998288,-0.157891,0.999276
hour,-0.028501,1.000000,0.066223,-0.079734,0.000561,0.002283,-0.000067,0.019228,-0.005986,0.218829,...,0.020288,-0.027997,0.015668,-0.027486,0.002982,-0.028479,0.015882,-0.028172,0.002904,-0.028273
day,-0.341420,0.066223,1.000000,-0.861325,-0.000722,-0.000992,0.001397,0.006744,0.078539,-0.083934,...,0.145789,-0.341330,0.038089,-0.334118,-0.304704,-0.344669,-0.019878,-0.335176,-0.444366,-0.344546
month,0.771245,-0.079734,-0.861325,1.000000,0.000913,0.002402,-0.000612,0.085946,-0.126763,-0.082048,...,-0.213301,0.770832,-0.191928,0.765319,0.105113,0.773057,-0.191293,0.766097,0.214928,0.772962
price,0.000808,0.000561,-0.000722,0.000913,1.000000,0.345061,0.240458,0.002088,-0.001417,-0.000084,...,0.001114,0.000763,-0.000428,0.000769,-0.000550,0.000766,-0.000580,0.000814,-0.000290,0.000746
distance,0.003290,0.002283,-0.000992,0.002402,0.345061,1.000000,0.025946,0.000984,-0.000415,-0.002738,...,0.000670,0.003298,-0.003192,0.003166,-0.003875,0.003318,-0.003649,0.003266,-0.003889,0.003319
surge_multiplier,0.000611,-0.000067,0.001397,-0.000612,0.240458,0.025946,1.000000,0.001471,-0.001961,-0.001594,...,-0.000090,0.000585,-0.000716,0.000594,-0.001998,0.000563,-0.001317,0.000502,-0.002214,0.000559
latitude,0.168295,0.019228,0.006744,0.085946,0.002088,0.000984,0.001471,1.000000,-0.531443,-0.098909,...,-0.047307,0.171976,-0.114437,0.169740,-0.109271,0.175336,-0.096327,0.171069,-0.082894,0.173359
longitude,-0.136716,-0.005986,0.078539,-0.126763,-0.001417,-0.000415,-0.001961,-0.531443,1.000000,0.012402,...,0.030330,-0.136620,0.048836,-0.140275,-0.007029,-0.136949,0.010131,-0.140399,-0.022605,-0.135525
temperature,-0.249027,0.218829,-0.083934,-0.082048,-0.000084,-0.002738,-0.001594,-0.098909,0.012402,1.000000,...,0.333160,-0.254755,0.789220,-0.242993,0.780426,-0.254172,0.817443,-0.231811,0.703078,-0.255578


In [ ]:
#Mengurutkan kolom yang memiliki nilai korelasi paling tinggi dengan kolom 'price' dengan nilai absolute 
df.corr().sort_values('price', ascending=False, key=abs).index

Index(['price', 'distance', 'surge_multiplier', 'latitude', 'moonPhase',
       'visibility.1', 'visibility', 'longitude', 'windBearing', 'humidity',
       'windGust', 'precipIntensityMax', 'windSpeed', 'month', 'cloudCover',
       'pressure', 'windGustTime', 'apparentTemperatureLow',
       'apparentTemperatureMinTime', 'timestamp', 'temperatureMinTime',
       'temperatureMaxTime', 'uvIndexTime', 'apparentTemperatureLowTime',
       'sunsetTime', 'sunriseTime', 'temperatureHighTime',
       'temperatureLowTime', 'apparentTemperatureMaxTime',
       'apparentTemperatureHighTime', 'day', 'dewPoint',
       'apparentTemperatureMin', 'hour', 'temperatureMax', 'temperatureHigh',
       'ozone', 'temperatureMin', 'uvIndex', 'apparentTemperatureMax',
       'apparentTemperatureHigh', 'precipProbability', 'apparentTemperature',
       'temperatureLow', 'precipIntensity', 'temperature'],
      dtype='object')

In [ ]:
# pengambilan kolom yang akan digunakan untuk model linear regresi
kolom=['price', 'distance', 'surge_multiplier','cab_type','name']
df_new= df[kolom]
df_new.head()

,price,distance,surge_multiplier,cab_type,name
0,5.0,0.44,1.0,Lyft,Shared
1,11.0,0.44,1.0,Lyft,Lux
2,7.0,0.44,1.0,Lyft,Lyft
3,26.0,0.44,1.0,Lyft,Lux Black XL
4,9.0,0.44,1.0,Lyft,Lyft XL


In [ ]:
df_new.isnull().sum()

price               0
distance            0
surge_multiplier    0
cab_type            0
name                0
dtype: int64

In [ ]:
df_new.head()

,price,distance,surge_multiplier,cab_type,name
0,5.0,0.44,1.0,Lyft,Shared
1,11.0,0.44,1.0,Lyft,Lux
2,7.0,0.44,1.0,Lyft,Lyft
3,26.0,0.44,1.0,Lyft,Lux Black XL
4,9.0,0.44,1.0,Lyft,Lyft XL


In [ ]:
#proses pengambilan data inference
data_inf= df_new.sample(10, random_state=21)

In [ ]:
# proses pemisahan data inference dengan data train test
data_train_test = df_new.drop(data_inf.index)
data_train_test

,price,distance,surge_multiplier,cab_type,name
0,5.0,0.44,1.0,Lyft,Shared
1,11.0,0.44,1.0,Lyft,Lux
2,7.0,0.44,1.0,Lyft,Lyft
3,26.0,0.44,1.0,Lyft,Lux Black XL
4,9.0,0.44,1.0,Lyft,Lyft XL
...,...,...,...,...,...
693065,9.5,1.00,1.0,Uber,WAV
693066,13.0,1.00,1.0,Uber,UberXL
693067,9.5,1.00,1.0,Uber,UberX
693069,27.0,1.00,1.0,Uber,Black SUV


In [ ]:
# printah untuk mereset index
data_train_test.reset_index(drop=True,inplace=True)
data_inf.reset_index(drop=True,inplace=True)

# Model Definition

In [ ]:
model_lin_reg =LinearRegression()

In [ ]:
# Hyper parameter yang digunakan adalah 'price'
y = data_train_test['price']
X = data_train_test.drop('price', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=True, random_state=21)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(478474, 4)
(159492, 4)


In [ ]:
linear = linear_model.LinearRegression()

# Model Training

In [ ]:
num_columns = X_train.select_dtypes(include=np.number).columns.tolist()
cat_columns = X_train.select_dtypes(include=['object']).columns.tolist()
print('Numeric Columns:', num_columns)
print('categorical Columns:', cat_columns)

Numeric Columns: ['distance', 'surge_multiplier']
categorical Columns: ['cab_type', 'name']


In [ ]:
X_train_num = X_train[num_columns]
X_train_cat = X_train[cat_columns]

X_test_num = X_test[num_columns]
X_test_cat = X_test[cat_columns]

In [ ]:
scaler = MinMaxScaler ()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled= scaler.transform(X_test_num)
X_train_num_scaled

array([[0.1875    , 0.        ],
       [0.06887755, 0.        ],
       [0.4005102 , 0.        ],
       ...,
       [0.30102041, 0.        ],
       [0.39413265, 0.        ],
       [0.22704082, 0.        ]])

In [ ]:
encoder =OrdinalEncoder()
X_train_cat_encoded = encoder.fit_transform(X_train_cat)
X_test_cat_encoded=encoder.transform(X_test_cat)
X_train_cat_encoded

array([[ 0.,  3.],
       [ 1., 10.],
       [ 1., 11.],
       ...,
       [ 1., 11.],
       [ 0.,  4.],
       [ 1.,  1.]])

In [ ]:
X_train_final = np.concatenate([X_train_num_scaled, X_train_cat_encoded], axis=1)
X_test_final = np.concatenate([X_test_num_scaled, X_test_cat_encoded], axis=1)
X_train_final_df=pd.DataFrame(X_train_final,columns=[num_columns+cat_columns])
X_train_final_df

,distance,surge_multiplier,cab_type,name
0,0.187500,0.0,0.0,3.0
1,0.068878,0.0,1.0,10.0
2,0.400510,0.0,1.0,11.0
3,0.075255,0.0,0.0,6.0
4,0.286990,0.0,1.0,11.0
...,...,...,...,...
478469,0.362245,0.0,1.0,9.0
478470,0.153061,0.0,1.0,0.0
478471,0.301020,0.0,1.0,11.0
478472,0.394133,0.0,0.0,4.0


In [ ]:
y.head()

0     5.0
1    11.0
2     7.0
3    26.0
4     9.0
Name: price, dtype: float64

In [ ]:
model_lin_reg =LinearRegression()

In [ ]:
model_lin_reg.fit(X_train_final, y_train)

LinearRegression()

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train_final,y_train)

print(model.intercept_)
print(model.coef_)

17.958302096257082
[21.89186853 40.8889688   2.31682495 -1.62177135]


In [ ]:
X_train_final_df.columns

MultiIndex([(        'distance',),
            ('surge_multiplier',),
            (        'cab_type',),
            (            'name',)],
           )

# Model evaluation

In [ ]:
# Evaluasi model menggunakan metode MSE
y_pred_train = model_lin_reg.predict(X_train_final)
y_pred_test = model_lin_reg.predict(X_test_final)

print('error train set',mean_squared_error(y_train,y_pred_train))
print('error test set',mean_squared_error(y_test,y_pred_test))

error train set 42.285476336625436
error test set 42.155588008973886


# Model Inference

In [ ]:
data_inf

,price,distance,surge_multiplier,cab_type,name
0,16.0,2.98,1.0,Uber,UberXL
1,19.5,1.72,1.0,Lyft,Lux Black
2,16.5,1.37,1.0,Lyft,Lux Black
3,8.0,1.54,1.0,Uber,UberPool
4,17.5,2.30,1.0,Uber,UberXL
5,11.5,1.50,1.0,Uber,WAV
6,11.5,0.85,1.0,Uber,WAV
7,7.5,1.41,1.0,Uber,WAV
8,10.5,4.71,1.0,Lyft,Shared
9,22.0,2.30,1.0,Uber,Black


In [ ]:
data_inf_num = data_inf[num_columns]
data_inf_cat = data_inf[cat_columns]

In [ ]:
data_inf_num_scaled = scaler.transform(data_inf_num)
data_inf_cat_encoded = encoder.transform(data_inf_cat)

In [ ]:
df_inf_final= np.concatenate([data_inf_num_scaled, data_inf_cat_encoded], axis=1)
df_inf_final

array([[ 0.37755102,  0.        ,  1.        , 10.        ],
       [ 0.21683673,  0.        ,  0.        ,  3.        ],
       [ 0.17219388,  0.        ,  0.        ,  3.        ],
       [ 0.19387755,  0.        ,  1.        ,  8.        ],
       [ 0.29081633,  0.        ,  1.        , 10.        ],
       [ 0.18877551,  0.        ,  1.        , 11.        ],
       [ 0.10586735,  0.        ,  1.        , 11.        ],
       [ 0.17729592,  0.        ,  1.        , 11.        ],
       [ 0.59821429,  0.        ,  0.        ,  7.        ],
       [ 0.29081633,  0.        ,  1.        ,  0.        ]])

In [ ]:
y_pred_inf= model_lin_reg.predict(df_inf_final)
y_pred_inf = pd.DataFrame(y_pred_inf, columns=['Price-Prediction'])
y_pred_inf

,Price-Prediction
0,12.322711
1,17.839949
2,16.862634
3,11.545298
4,10.423926
5,6.568291
6,4.753276
7,6.316981
8,19.701931
9,26.641640


In [ ]:
pd.concat([data_inf, y_pred_inf], axis=1)

,price,distance,surge_multiplier,cab_type,name,Price-Prediction
0,16.0,2.98,1.0,Uber,UberXL,12.322711
1,19.5,1.72,1.0,Lyft,Lux Black,17.839949
2,16.5,1.37,1.0,Lyft,Lux Black,16.862634
3,8.0,1.54,1.0,Uber,UberPool,11.545298
4,17.5,2.30,1.0,Uber,UberXL,10.423926
5,11.5,1.50,1.0,Uber,WAV,6.568291
6,11.5,0.85,1.0,Uber,WAV,4.753276
7,7.5,1.41,1.0,Uber,WAV,6.316981
8,10.5,4.71,1.0,Lyft,Shared,19.701931
9,22.0,2.30,1.0,Uber,Black,26.641640


# Kesimpulan

Parameter numerik yang digunakan untuk persamaan regresi:
- Kategori : cab_type & name 
- Numeric : distance & surge_multiplier

Bentuk persamaan linear regresi untuk Y= price adalah
- y= 17.96049043105303 + 21.87513737distance + 40.73327338surge_multiplier + 2.32171449cab_type - 1.6217196name
disederhanakan
- y = 18 + 22distance + 41surge_multiplier + 2.4cab_type - 1.6name

bisa diambil kesimpulan dari data yang sudah di scaled dan juga encoded bahwa parameter yang paling berpengaruh terhadap price adalah 'surge multiplier' sementara parameter yang paling tidak berpengaruh adalah 'name'

- Linear regresi ini memiliki nilai error sebesar 42 % hal ini dikarenakan parameter yang digunakan untuk memprediksi harga hanya 4 dari 57 paramter dari data set awal